In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [2]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        # TODO: Implement the predict method
        predictions = []
        for x in X:
            distances = [self.compute_distance(x, x_train) for x_train in self.X_train]
            k_nearest_neighbors = np.argsort(distances)[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_nearest_neighbors]
            predictions.append(np.mean(k_nearest_labels))  # Probability of churn
        return np.array(predictions)

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1 - X2) ** 2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1 - X2))

In [3]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # TODO: Implement data preprocessing
    # Handle categorical variables, scale features, etc.
    # Drop unnecessary columns
    train_data.drop(['id', 'CustomerId', 'Surname'], axis=1, inplace=True)
    test_data.drop(['id', 'CustomerId', 'Surname'], axis=1, inplace=True)
    
    # Handle categorical variables (Geography, Gender)
    le = LabelEncoder()
    train_data['Geography'] = le.fit_transform(train_data['Geography'])
    test_data['Geography'] = le.transform(test_data['Geography'])
    
    train_data['Gender'] = le.fit_transform(train_data['Gender'])
    test_data['Gender'] = le.transform(test_data['Gender'])

    # Separate features and target
    X_train = train_data.drop('Exited', axis=1).values
    y_train = train_data['Exited'].values
    X_test = test_data.values

    # Scale the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, y_train, X_test

In [4]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    # Compute ROC AUC scores
    kf = KFold(n_splits=n_splits)
    scores = []
    
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_val)
        
        auc_score = roc_auc_score(y_val, y_pred)
        scores.append(auc_score)
    
    return np.mean(scores)

In [5]:
# Load and preprocess data
X, y, X_test = preprocess_data('train.csv', 'test.csv')

# # Create and evaluate model
# knn = KNN(k=5, distance_metric='euclidean')

# # Perform cross-validation
# cv_scores = cross_validate(X, y, knn)

# print("Cross-validation scores:", cv_scores)

# TODO: hyperparamters tuning
def hyperparameter_tuning(X, y):
    # Define the hyperparameter grid
    k_values = [3, 5, 7, 9, 11]
    distance_metrics = ['euclidean', 'manhattan']
    
    best_score = 0
    best_params = {'k': None, 'distance_metric': None}
    
    for k in k_values:
        for distance_metric in distance_metrics:
            knn = KNN(k=k, distance_metric=distance_metric)
            cv_score = cross_validate(X, y, knn)
            print(f"CV Score with k={k}, distance_metric={distance_metric}: {cv_score}")
            
            # Update best params if current score is better
            if cv_score > best_score:
                best_score = cv_score
                best_params['k'] = k
                best_params['distance_metric'] = distance_metric
    
    print(f"Best params: {best_params}, Best CV Score: {best_score}")
    return best_params

best_params = hyperparameter_tuning(X, y)

# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
knn = knn = KNN(k=best_params['k'], distance_metric=best_params['distance_metric'])
knn.fit(X, y)
test_predictions = knn.predict(X_test)

# Save test predictions
pd.DataFrame({'id': pd.read_csv('test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

CV Score with k=3, distance_metric=euclidean: 0.845940045176758
CV Score with k=3, distance_metric=manhattan: 0.8487764271379048


KeyboardInterrupt: 